<a href="https://colab.research.google.com/github/damianperreira/BigData/blob/master/data_challenge_M16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

In [3]:
# driver to allow spark to interact with pg
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-02-05 14:24:48--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.73MB/s    in 0.2s    

2020-02-05 14:24:48 (4.73 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [0]:
# Start session with additional option that adds driver to spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataHW").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# https://damianperreira-bucket.s3.amazonaws.com/amazon_reviews_us_Video_Games_v1_00.tsv
# https://damianperreira-bucket.s3.amazonaws.com/amazon_reviews_us_Watches_v1_00.tsv

# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://damianperreira-bucket.s3.amazonaws.com/amazon_reviews_us_Video_Games_v1_00.tsv"
spark.sparkContext.addFile(url)
games_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv"), sep="\t", header=True)

games_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [6]:
# https://damianperreira-bucket.s3.amazonaws.com/amazon_reviews_us_Video_Games_v1_00.tsv
# https://damianperreira-bucket.s3.amazonaws.com/amazon_reviews_us_Watches_v1_00.tsv

# Read in data from S3 Buckets
from pyspark import SparkFiles
url2 ="https://damianperreira-bucket.s3.amazonaws.com/amazon_reviews_us_Watches_v1_00.tsv"
spark.sparkContext.addFile(url2)
watches_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv"), sep="\t", header=True)

watches_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [7]:
# Drop null values
watches_dropna_df = watches_data_df.dropna()
watches_dropna_df.show()

games_dropna_df = games_data_df.dropna()
games_dropna_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [0]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [9]:
# Filter for only data where the marketplace is "US"
watches_cleaned_df = watches_dropna_df.filter(col("marketplace") == 'US')
watches_cleaned_df.show()

games_cleaned_df = games_dropna_df.filter(col("marketplace") == 'US')
games_cleaned_df.show()

union_df = watches_cleaned_df.unionAll(games_cleaned_df).distinct()

union_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [0]:
# Configure settings for RDS - #//dataviz.cja19o8tndmt.us-east-1.rds.amazonaws.com:5432/<database name>"

mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cja19o8tndmt.us-east-1.rds.amazonaws.com:5432/data_challenge_MXVI"
config = {"user":"postgres",
          "password": "zion2874",
          "driver":"org.postgresql.Driver"}

In [11]:
# Create a dataframe to match the customer table
customers_count = union_df.groupby('customer_id').count()
customers_tbl = customers_count.withColumnRenamed("count", "customer_count")

customers_tbl.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   23219103|             3|
|    6302672|             1|
|   23578446|             1|
|   10525605|             1|
|    1460356|             2|
|   15081523|             1|
|   30258719|             2|
|   10979021|             1|
|   19633372|             6|
|    9177020|             5|
|   42481884|             1|
|   13934414|             1|
|   38449172|             5|
|   52198158|             6|
|   37402721|             1|
|   14940986|             1|
|   46531665|             1|
|   49961139|             1|
|   52752630|             7|
|   40309779|             1|
+-----------+--------------+
only showing top 20 rows



In [0]:
# Write DataFrame to the customers table in RDS
customers_tbl.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [20]:
# Remove product key duplicates
products_tbl = union_df.select("product_id","product_title").dropDuplicates()

products_tbl.show(truncate= False)

+----------+---------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                  |
+----------+---------------------------------------------------------------------------------------------------------------+
|B00NL1CZ1C|Nuband. Activ+ Activity Tracker                                                                                |
|B009BEO81I|Fossil Riley                                                                                                   |
|B000UMI0FK|Armitron Sport Women's 25-6355PNK Pink and Silver-Tone Easy to Read Watch                                      |
|B0083XFT5M|Timex Unisex T2N893 Weekender Silver-Tone Watch with Leather Band                                              |
|B00N1Y8TQ4|Tissot Men's T095.417.36.057.01 'Quickster' Black Dial Black Leather Strap Chronograph Swiss Quartz Watch      |


In [0]:
# Write DataFrame to the products table in RDS
products_tbl.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [28]:
# Review_id table

# Remove duplicates
review_id_table = union_df.select("review_id","customer_id","product_id","product_parent","review_date").dropDuplicates()

review_id_table.show(truncate=False)

+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R16ILL36QLL21N|15479306   |B004YZIBRG|932904845     |2014-11-11 |
|R1JZ8NWX4NAM1E|40092040   |B00J05Z69G|983164742     |2014-11-02 |
|R1NTZ0QA4TMZ9K|22214743   |B00CZ5VF84|955324959     |2014-11-25 |
|R1YW5ZR0NRTY5N|14140253   |B0071M6J64|871304516     |2014-09-19 |
|R2HU9I9YLJQVC3|2006130    |B000FVE3BG|824370661     |2015-05-29 |
|R2I6A4L32MBJY3|21850063   |B003ERFEGG|580698028     |2014-10-05 |
|R2OTB6MXYA7U8M|16513355   |B00FWVI8PQ|413132443     |2015-07-03 |
|R2PWXZT17SVTO6|42018641   |B004GHRESY|677543212     |2015-02-18 |
|R2UZ6WHSA56977|14042720   |B000ROGWI8|311585226     |2015-07-10 |
|R34GEEP17FWX1A|16002650   |B00Q8I9JCS|630301844     |2015-05-04 |
|R39DC6BFIM0ODW|1202083    |B00E6X3BFS|487408803     |2015-03-21 |
|RQKIOEOX3DM2C |34102393   |B005V4OM5O|463993069     |2014-11-

In [0]:
# Write DataFrame to the review_id_table table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [32]:
# vine table

# Remove duplicates
vine_table = union_df.select("review_id","star_rating","helpful_votes","total_votes","vine").dropDuplicates()

vine_table.show(truncate=False)

+--------------+-----------+-------------+-----------+----+
|review_id     |star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1KP8QCE4CJ5P5|5          |0            |0          |N   |
|R27HG7FGOS7OXU|5          |0            |0          |N   |
|R2IL5B7KSRMJ7L|3          |0            |0          |N   |
|R2O2WCYCRCW6KH|4          |0            |0          |N   |
|R2PWXZT17SVTO6|5          |0            |0          |N   |
|R2Y8U8C2OU701S|5          |0            |0          |N   |
|R33ENKD2BS8ML8|4          |3            |4          |N   |
|R3H4KR95LHY1SC|1          |2            |2          |N   |
|R3PTHDS99BM3RH|5          |0            |0          |N   |
|RJIDX4BNG4YF6 |5          |0            |0          |N   |
|ROF160W28LK8K |2          |1            |2          |N   |
|RZTQN04E0H4LT |4          |1            |1          |N   |
|R14MUAAO2UZTW2|3          |0            |0          |N   |
|R1GM0299RZYJ8E|5          |3           

In [0]:
# Write DataFrame to the vine table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

## ✓ Come up with a conclusion on the trustworthiness of vine reviews with data to back up their claim

I initially thought the vine reviewes may have some bias due to the fact that they were received for free on compensated in some way.

After comparing the average star ratings for the watches and games dataset I found that there's very little variance between the two at about 4 stars overall.

I also found that over 60% of the ratings for both Vine and Non-Vine ratings are 4 and 5 star and helpful as well.

